# Отчет по лабораторной работе 2
## Неполнодоступная двухсервисная модель Эрланга с одинаковыми интенсивностями обслуживания и зарезервированной емкостью

Генералов Даниил, 1032212280

# 0. теоретическая информация

Исследуется сота сети связи емкостью $C$. Пусть пользователям сети предоставляются услуги двух
типов. Запросы в виде двух пуассоновский потоков (ПП) с интенсивностями $\lambda_1$, $\lambda_2$ поступают
в соту. Среднее время обслуживания запросов на предоставление услуг каждого типа $µ_1^{-1}$, $µ_2^{-1}$
соответственно. Исследуются основные характеристики модели для случая $\mu_1 = \mu_2 = \mu$.

Часть пропускной способности соты зарезервирована для обслуживания запросов на
предоставление услуги 1-го или 2-го типа. Оставшаяся часть пропускной способности является
полнодоступной для запросов на предоставление услуг обоих типов. Предположим, что сначала
заполняется полнодоступная емкость.

В классификации Башарина-Кендалла: $M (\lambda_1) M (\lambda_2) | M (\mu_1) M (\mu_2) |C, g|0$.

-  $C$ -- пиковая пропускная способность соты;
- $g$ -- полнодоступная часть пропускной способности соты;
- $C - g$ -- пропускная способность, зарезервированная для обслуживание запросов определенного типа;
-  $\lambda_1$, $\lambda_2$ -- интенсивность поступления запросов на предоставление услуги 1, 2-го типа;
-  $\mu^{-1}$ -- среднее время обслуживания запроса на предоставление услуги 1, 2-го типа;
-  $\rho_1$, $\rho_2$ -- интенсивность предложенной нагрузки, создаваемой запросами на предоставление услуги 1, 2-го типа;
-  $X(t)$  --   число запросов, обслуживаемых в системе в  момент времени $t$, $t \geq 0$ (случайный процесс (СП), описывающий функционирование системы в момент времени $t$, $t \geq 0$);
-  $X$ -- пространство состояний системы;
-  $n$ -- число обслуживаемых в системе запросов;
-  $B_1$, $B_2$ -- множество блокировок запросов на предоставление услуги 1, 2-го типа;
-  $S_1$, $S_2$ -- множество приема запросов на предоставление услуги 1, 2-го типа.

Пусть часть пропускной способности соты $С-g$ зарезервирована для обслуживания запросов
на предоставление услуги 1-го типа.

Пространство состояний системы: $X = \{0, ..., C\}$; $|X| = C+1$

Множество блокировок запросов на предоставление услуги $i$-типа, $i = 1,2$:
  $B_1 \{C\}$; $B_2 = \left\{g, g+1, ..., C\right\}$

Множество приема запросов на предоставление услуги $i$-типа, $i = 1,2$:
  $S_i = \overline{B_i} = X \setminus B_i$; $S_1 = \{0, ..., C-1\}$; $S_2 = \{0, ..., C_2-1\}$

СУГБ:

- $(\lambda_1 + \lambda_2)p_0 = \mu p_1$
- $(\lambda_1 + \lambda_2 + n \mu) p_n = (\lambda_1 + \lambda_2)p_{n-1} + (n+1)\mu p_{n+1}$, $n=\overline{1, g-1}$
- $(\lambda_1 + g\mu)p_g = (\lambda_1 + \lambda_2)p_{g-1} + (g+1)\mu p_{g+1}$
- $(\lambda_1 + n\mu)p_n = \lambda_1 p_{n-1} + (n+1)\mu p_{n+1}$, $n = \overline{g+1, C-1}$
- $C \mu p_C = \lambda_1 p_{C-1}$

СУЛБ:
- $(\lambda_1 + \lambda_2) p_1 = n \mu p_n$, $n = \overline{1,g}$
- $\lambda_1 p_{n-1} = n \mu p_n$, $n = \overline{g+1, C}$

Обозначим $\rho_1 = \frac{\lambda_1}{\mu}$, $\rho_2 = \frac{\lambda_2}{\mu}$.

Стационарное распределение вероятностей состояний системы:

$p_n =$
- $p_0 * \frac{(\rho_1 + \rho_2)^n}{n!}$, если $n = \overline{1,g}$;
- $p_0 * \frac{(\rho_1 + \rho_2)^g * (\rho_1)^{n-g}}{n!}$, если $n = \overline{g+1, C}$.

При этом $p_0 = (\sum_{n=1}^g \frac{(\rho_1 + \rho_2)^n}{n!} + \sum_{n=g+1}^C \frac{(\rho_1 + \rho_2)^g * (\rho_1)^{n-g}}{n!}) ^ {-1}$

Основные вероятностные характеристики модели:

- Вероятность блокировки по времени запроса на предоставление услуги 1-го типа: $E_1 = \sum_{n \in B_1} p_n = p_C$
- Вероятность блокировки по времени запроса на предоставление услуги 1-го типа: $E_2 = \sum_{n \in B_2} p_n = p_g + p_{g+1} + ... + p_C = \sum_{n=g}^C p_n$
- Среднее число обслуживаемых в системе запросов: $\overline{N} = \sum_{n \in X} np_n$

## 1. подключение библиотек, определение функций

Для расчета больших факториалов нам потребуется длинная арифметика, а для рисования графиков -- библиотека для визуализации данных.

In [2]:
:dep num = { version = "^0.4.3" }
:dep plotters = { version = "^0.3.6", default-features = false, features = ["evcxr", "all_series", "all_elements"] }

extern crate num;
use num::BigRational as R;
use num::BigInt as I;
use num::BigUint as U;
use num::Integer;
use num::traits::ConstZero;
use num::FromPrimitive;
use num::ToPrimitive;

extern crate plotters;
use plotters::prelude::*;

Для удобства конвертации стандартных чисел в числа длинной арифметики используются helper-функции.

In [3]:
fn u(i: usize) -> U {
    U::from_usize(i).unwrap()
}

fn rr(i: f64) -> R {
    R::from_float(i).unwrap()
}

Для вычисления факториала нет стандартной функции, и очевидные подходы не работают с длинной арифметикой, поэтому эта функция считает это за нас.

Мы также проверяем, работает ли одно из правил факториалов: что $N! = (N-i)! + \prod_{x=N-i+1}^N x$.

In [4]:

use num::{BigInt, BigUint};
use num::ToPrimitive;
fn factorial(n: u8) -> BigUint {
    (1..=n).map(BigUint::from).product()
}

for i in 3..8 {
    assert_eq!(factorial(10), factorial(i) * (i+1 ..= 10).map(|x| x as usize).product::<usize>());
}

()

Эта функция отображает график функции, принимая на вход список X-Y пар.

In [5]:
fn draw_chart(data: &Vec<(f32, f32)>, name: impl ToString) -> plotters::evcxr::SVGWrapper {
    let minx = data.iter().min_by(|a, b| a.0.partial_cmp(&b.0).unwrap_or(std::cmp::Ordering::Equal)).unwrap().0;
    let maxx = data.iter().max_by(|a, b| a.0.partial_cmp(&b.0).unwrap_or(std::cmp::Ordering::Equal)).unwrap().0;
    let miny = data.iter().min_by(|a, b| a.1.partial_cmp(&b.1).unwrap_or(std::cmp::Ordering::Equal)).unwrap().1;
    let maxy = data.iter().max_by(|a, b| a.1.partial_cmp(&b.1).unwrap_or(std::cmp::Ordering::Equal)).unwrap().1;
    let figure = evcxr_figure((640, 480), |root| {
        root.fill(&WHITE)?;
        let mut chart = ChartBuilder::on(&root)
            .caption(name.to_string(), ("Arial", 50).into_font())
            .margin(5)
            .x_label_area_size(30)
            .y_label_area_size(30)
            .build_cartesian_2d(minx..maxx, miny..maxy)?;

        chart.configure_mesh().draw()?;

        chart.draw_series(LineSeries::new(
            data.clone(),
            &RED,
        )).unwrap();

        // chart.configure_series_labels()
        //     .background_style(&WHITE.mix(0.8))
        //     .border_style(&BLACK)
        //     .draw()?;
        Ok(())
    });
    return figure;
}


Эта функция считает стационарное распределение вероятностей для рассматриваемой модели.
Она принимает $\rho_1$ и $\rho_2$, которые соответствуют интенсивностям поступления заявок на два типа услуг,
а также $C$ -- максимальную пропускную способность, а именно количество заявок, которые могут одновременно выполняться,
и $g$ -- зарезервированная емкость для заявок первого типа.

Подсчет разделен на два случая: `prob_low` и `prob_high` -- первое относится к случаю, когда $i < g$, а второе к остальным случаям. Из-за этого `prob_low` используется в определении `prob_high`.

Функция возвращает список: $i$-й элемент списка равен вероятности, что система будет найдена в состоянии $i$ (то есть это $p_i$).

In [6]:
/// Считает стационарное распределение вероятностей для пространства
fn stationary_prob_distribution(rho1: f64, rho2: f64, max_c: u8, g: u8) -> Vec<f64> {
    let prob_low = |x| (rho1 + rho2).powf(x as f64) / (factorial(x).to_f64().unwrap());
    let prob_high = |x| (
        prob_low(g) *
        rho1.powi(x as i32 - g as i32) /
        ((g+1) as usize..=x as usize).map(|x| x as f64).product::<f64>()
    );
    let prob = |x| if x<=g {prob_low(x)} else {prob_high(x)};

    let mut v = Vec::with_capacity(max_c as usize);
    let p0 = 1.0/(1..=max_c)
        // .map(|x| (println!("{x}"), x).1)
        .map(prob).sum::<f64>();

    for c in 0..=max_c {
        let res = p0 * prob(c);
        v.push(res);
    }
    v
}

# 2. входные параметры

Здесь задаются параметры, которые определяют модель.
Чтобы попробовать запустить вычисления с другими значениями, вы можете поменять эту ячейку и перезапустить ее и все ячейки ниже.

При разработке я использовал следующие значения для теста:

- $\lambda1 = 80$
- $\lambda2 = 50$
- $\mu_1 = \mu_2 = 2$
- $C = 60$
- $g = 40$

In [7]:
let lambda1 = 80.0;
let lambda2 = 50.0;
let mu = 2.0;
let mu1 = mu;
let mu2 = mu;
let rho1 = lambda1 / mu1;
let rho2 = lambda2 / mu2;
let c = 60;
let g: usize = 40;

# 3. расчет
Распределение вероятностей можно посчитать с помощью нашей функции сверху.
По свойству распределения вероятностей, сумма всех элементов должна быть равна 1, и мы здесь проверяем это.

In [9]:
let dist = stationary_prob_distribution(rho1, rho2, c as u8, g as u8);
println!("probability distribution: {dist:?}");
println!("Sum should be 1: {}", dist.iter().sum::<f64>());

probability distribution: [2.5542482810751626e-26, 1.6602613826988556e-24, 5.395849493771281e-23, 1.1691007236504442e-21, 1.899788675931972e-20, 2.469725278711563e-19, 2.6755357186041938e-18, 2.4844260244181796e-17, 2.018596144839771e-16, 1.4578749934953903e-15, 9.476187457720037e-15, 5.599565315925476e-14, 3.0330978794596327e-13, 1.5165489397298164e-12, 7.041120077317004e-12, 3.0511520335040355e-11, 1.2395305136110144e-10, 4.739381375571525e-10, 1.7114432745119397e-9, 5.854937518067162e-9, 1.9028546933718278e-8, 5.889788336627086e-8, 1.7401647358216393e-7, 4.917856862104632e-7, 1.3319195668200045e-6, 3.4629908737320113e-6, 8.657477184330028e-6, 2.084207470301674e-5, 4.838338770343171e-5, 0.00010844552416286419, 0.00023496530235287238, 0.000492669182352797, 0.001000734276654119, 0.001971143272197507, 0.0037683621380246446, 0.0069983868277600555, 0.01263597621678899, 0.022198336597061736, 0.03797083891602666, 0.0632847315267111, 0.10283768873090554, 0.10032945242039565, 0.09555185944799

Среднее число запросов, которые находятся в обработке -- это математическое ожидание набора вероятностей: ((вероятность того, что там 1 заявка) * 1 + (вероятность того, что там 2 заявки) * 2 + ...) / (макс. заявок)

Чтобы посчитать это, мы считаем сумму значений вероятности, помноженных на свой индекс (который равен количеству заявок, соответствующих этой ячейке).

In [10]:
let avg: f64 = dist.iter().enumerate().map(|(i,v)| (i as f64) * v).sum();
println!("Average requests in flight: {avg}");



Average requests in flight: 43.72435164097261


Вероятность блокировки по времени -- это вероятность того, что система оказывается в состоянии, когда нельзя обработать ни одну заявку, пока нагрузка не спадет.

Для заявок первого типа это -- только случай, когда нет больше ячеек вообще (то есть $p_C$), а для второго типа это все случаи, когда обрабатывается больше чем $g$ заявок (потому что та пропускная способность зарезервирована для заявок первого типа).

In [11]:
let time_blocking_prob1 = dist.last().copied().unwrap();
let time_blocking_prob2 = dist.iter().skip(g).sum::<f64>();
println!("time blocking probability (E1): {time_blocking_prob1}");
println!("time blocking probability (E2): {time_blocking_prob2}");


time blocking probability (E1): 0.001108721202201012


time blocking probability (E2): 0.8492519804375737


Вероятность блокировки по вызовам -- это вероятность, что определенный из двух типов запросов будет заблокирован;
поэтому это связано с вероятностью поступления каждого из двух запросов.

In [12]:
let req_blocking_prob1 = lambda1 / (lambda1 + lambda2) * time_blocking_prob1;
let req_blocking_prob2 = lambda2 / (lambda1 + lambda2) * time_blocking_prob1;
println!("request blocking probability: {req_blocking_prob1}, {req_blocking_prob2}");

request blocking probability: 0.0006822899705852383, 0.0004264312316157739


# 4. графики

График распределения вероятности показывает, какие состояния системы являются самыми частыми.
Если пик этого графика находится на правой границе, значит система перегружена;
иначе он показывает среднее количество ожидающих своей очереди запросов (таким образом это значение имеет похожий смысл на $E$).

In [13]:


draw_chart(&dist.iter().enumerate().map(|(x,y)| (x as f32,*y as f32)).collect(), "n vs p_n")

n vs p_n
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.01
 
 
 
0.02
 
 
 
0.03
 
 
 
0.04
 
 
 
0.05
 
 
 
0.06
 
 
 
0.07
 
 
 
0.08
 
 
 
0.09
 
 
 
0.1
 
 
 
 
0.0
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,444 44,444 54,444 64,444 74,444 84,444 94,444 104,444 114,444 124,444 134,444 144,444 154,444 164,444 174,444 184,444 194,444 204,444 214,444 224,444 234,444 244,444 254,444 264,444 274,444 284,444 294,444 304,444 314,444 324,444 334,444 344,443 354,441 364,437 374,430 384,418 394,397 404,361 414,301 424,206 434,56 444,66 454,84 464,109 474,140 484,174 494,209 504,244 514,277 524,308 534,335 544,359 554,379 564,395 574,408 584,418 594,425 604,431 614,435 624,438 634,440 "/>

Для того, чтобыв посчитать график зависимости вероятности блокировки от интенсивности,
надо зафиксировать значение одной из переменных $\lambda$ и варьировать другую;
для каждой такой конфигурации нужно вычислить значение E
и затем отобразить на графике.

Сначала делаем это для $\lambda_1$.

In [15]:
let mut block_prob = vec![];
for lambda1 in 1..=100 {
    let rho1 = lambda1 as f64 / &mu1;
    let dist = stationary_prob_distribution(rho1, rho2, c as u8, g as u8);
    block_prob.push((lambda1 as f32, dist.last().cloned().unwrap() as f32));
}
block_prob

[(1.0, 1.77e-43), (2.0, 2.4478834e-37), (3.0, 1.0591664e-33), (4.0, 4.286179e-31), (5.0, 4.7081145e-29), (6.0, 2.2570824e-27), (7.0, 6.0856095e-26), (8.0, 1.0738625e-24), (9.0, 1.3677995e-23), (10.0, 1.3447435e-22), (11.0, 1.0705925e-21), (12.0, 7.151608e-21), (13.0, 4.1184912e-20), (14.0, 2.0883733e-19), (15.0, 9.482566e-19), (16.0, 3.9085558e-18), (17.0, 1.4789452e-17), (18.0, 5.1855205e-17), (19.0, 1.6980775e-16), (20.0, 5.2282743e-16), (21.0, 1.5222797e-15), (22.0, 4.212393e-15), (23.0, 1.112621e-14), (24.0, 2.8158101e-14), (25.0, 6.851038e-14), (26.0, 1.6073124e-13), (27.0, 3.6457583e-13), (28.0, 8.014026e-13), (29.0, 1.7108645e-12), (30.0, 3.5540039e-12), (31.0, 7.196365e-12), (32.0, 1.4226139e-11), (33.0, 2.7495598e-11), (34.0, 5.20248e-11), (35.0, 9.64826e-11), (36.0, 1.7557211e-10), (37.0, 3.1381067e-10), (38.0, 5.5142735e-10), (39.0, 9.534293e-10), (40.0, 1.6233456e-9), (41.0, 2.723784e-9), (42.0, 4.5067874e-9), (43.0, 7.3581092e-9), (44.0, 1.1861042e-8), (45.0, 1.888738e-8),

In [16]:
draw_chart(&block_prob, "lambda1 vs E")

lambda1 vs E
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.002
 
 
 
0.004
 
 
 
0.006
 
 
 
0.008
 
 
 
0.01
 
 
 
0.012
 
 
 
0.014
 
 
 
0.016
 
 
 
0.018
 
 
 
0.02
 
 
 
0.022
 
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,444 41,444 47,444 53,444 59,444 65,444 71,444 77,444 83,444 89,444 95,444 101,444 107,444 113,444 119,444 125,444 131,444 137,444 143,444 149,444 156,444 162,444 168,444 174,444 180,444 186,444 192,444 198,444 204,444 210,444 216,444 222,444 228,444 234,444 240,444 246,444 252,444 258,444 264,444 270,444 277,444 283,444 289,444 295,444 301,444 307,444 313,444 319,444 325,444 331,444 337,444 343,444 349,444 355,444 361,444 367,444 373,444 379,444 385,444 391,444 398,444 404,444 410,444 416,444 422,444 428,444 434,444 440,443 446,443 452,443 458,442 464,441 470,440 476,439 482,438 488,436 494,434 500,432 506,429 512,426 519,421 525,416 531,411 537,404 543,396 549,387 555,376 561,364 567,350 573,335 579,318 585,298 591,276 597,253 603,226 609,198 615,166 621,132 627,96 634,56 "/>

После этого -- для $\lambda_2$.

In [17]:
let mut block_prob = vec![];
for lambda2 in 1..=100 {
    let rho2 = lambda2 as f64 / &mu1;
    let dist = stationary_prob_distribution(rho1, rho2, c as u8, g as u8);
    block_prob.push((lambda2 as f32, dist.last().cloned().unwrap() as f32));
}
block_prob

[(1.0, 0.0007004019), (2.0, 0.0007204047), (3.0, 0.0007394996), (4.0, 0.00075771636), (5.0, 0.0007750877), (6.0, 0.00079164817), (7.0, 0.0008074333), (8.0, 0.0008224789), (9.0, 0.0008368207), (10.0, 0.0008504938), (11.0, 0.00086353236), (12.0, 0.0008759696), (13.0, 0.0008878374), (14.0, 0.0008991663), (15.0, 0.00090998545), (16.0, 0.0009203226), (17.0, 0.00093020406), (18.0, 0.00093965477), (19.0, 0.00094869814), (20.0, 0.00095735653), (21.0, 0.0009656507), (22.0, 0.0009736004), (23.0, 0.0009812242), (24.0, 0.0009885395), (25.0, 0.0009955628), (26.0, 0.0010023093), (27.0, 0.0010087935), (28.0, 0.001015029), (29.0, 0.0010210287), (30.0, 0.0010268046), (31.0, 0.0010323678), (32.0, 0.0010377292), (33.0, 0.0010428985), (34.0, 0.0010478852), (35.0, 0.0010526981), (36.0, 0.0010573457), (37.0, 0.0010618357), (38.0, 0.0010661754), (39.0, 0.0010703718), (40.0, 0.0010744317), (41.0, 0.0010783611), (42.0, 0.0010821657), (43.0, 0.0010858513), (44.0, 0.0010894231), (45.0, 0.001092886), (46.0, 0.001

In [18]:
draw_chart(&block_prob, "lambda2 vs E")

lambda2 vs E
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.00075
 
 
 
0.0008
 
 
 
0.00085
 
 
 
0.0009
 
 
 
0.00095
 
 
 
0.001
 
 
 
0.00105
 
 
 
0.0011
 
 
 
0.00115
 
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,444 41,429 47,414 53,399 59,386 65,373 71,360 77,348 83,337 89,326 95,316 101,306 107,297 113,288 119,280 125,272 131,264 137,256 143,249 149,242 156,236 162,230 168,224 174,218 180,212 186,207 192,202 198,197 204,192 210,188 216,183 222,179 228,175 234,171 240,167 246,164 252,160 258,157 264,154 270,150 277,147 283,144 289,141 295,139 301,136 307,133 313,131 319,128 325,126 331,123 337,121 343,119 349,117 355,115 361,113 367,111 373,109 379,107 385,105 391,103 398,101 404,100 410,98 416,96 422,95 428,93 434,92 440,90 446,89 452,87 458,86 464,85 470,83 476,82 482,81 488,80 494,78 500,77 506,76 512,75 519,74 525,73 531,72 537,71 543,70 549,69 555,68 561,67 567,66 573,65 579,64 585,63 591,62 597,61 603,61 609,60 615,59 621,58 627,57 634,56 "/>

Для того, чтобы нарисовать график среднего числа обслуживаемых запросов, нужно сделать то же самое:
по разным значениям $\lambda$ посчитать распределение вероятности,
из него -- среднее количество,
и отобразить для него точку.
Сначала делаем это, варьируя $\lambda_1$, а затем $\lambda_2$.

In [19]:
let mut avg_req_counts = vec![];
for lambda1 in 1..=100 {
    let rho1 = (lambda1 as f64) / &mu1;
    let dist = stationary_prob_distribution(rho1, rho2, c as u8, g as u8);
    let avg: f64 = dist.iter().enumerate().map(|(i,v)| i as f64 * v).sum();
    avg_req_counts.push((lambda1 as f32, avg as f32));
}
avg_req_counts

[(1.0, 25.452143), (2.0, 25.93603), (3.0, 26.415709), (4.0, 26.890446), (5.0, 27.35946), (6.0, 27.821955), (7.0, 28.277117), (8.0, 28.724144), (9.0, 29.162264), (10.0, 29.590746), (11.0, 30.00892), (12.0, 30.416199), (13.0, 30.812069), (14.0, 31.196117), (15.0, 31.568027), (16.0, 31.927584), (17.0, 32.274662), (18.0, 32.60924), (19.0, 32.93138), (20.0, 33.241222), (21.0, 33.538982), (22.0, 33.824936), (23.0, 34.09942), (24.0, 34.3628), (25.0, 34.615498), (26.0, 34.85795), (27.0, 35.09061), (28.0, 35.313957), (29.0, 35.528473), (30.0, 35.734642), (31.0, 35.932945), (32.0, 36.123863), (33.0, 36.307865), (34.0, 36.485416), (35.0, 36.656967), (36.0, 36.822956), (37.0, 36.983807), (38.0, 37.139935), (39.0, 37.291744), (40.0, 37.439613), (41.0, 37.583927), (42.0, 37.72504), (43.0, 37.863308), (44.0, 37.99907), (45.0, 38.13266), (46.0, 38.264397), (47.0, 38.394592), (48.0, 38.523563), (49.0, 38.6516), (50.0, 38.779007), (51.0, 38.90607), (52.0, 39.03308), (53.0, 39.160324), (54.0, 39.288082),

In [20]:
draw_chart(&avg_req_counts, "lambda1 vs Nbar")

lambda1 vs Nbar
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
30.0
 
 
 
35.0
 
 
 
40.0
 
 
 
45.0
 
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,444 41,437 47,429 53,421 59,414 65,406 71,399 77,392 83,385 89,378 95,371 101,364 107,358 113,352 119,346 125,340 131,335 137,329 143,324 149,319 156,314 162,310 168,305 174,301 180,297 186,293 192,289 198,286 204,282 210,279 216,276 222,272 228,270 234,267 240,264 246,261 252,259 258,256 264,254 270,251 277,249 283,247 289,244 295,242 301,240 307,238 313,236 319,234 325,232 331,230 337,228 343,226 349,224 355,221 361,219 367,217 373,215 379,213 385,211 391,209 398,206 404,204 410,202 416,199 422,197 428,194 434,192 440,189 446,186 452,184 458,181 464,178 470,175 476,171 482,168 488,165 494,161 500,158 506,154 512,150 519,146 525,142 531,138 537,133 543,129 549,125 555,120 561,115 567,111 573,106 579,101 585,96 591,91 597,86 603,81 609,76 615,71 621,66 627,61 634,56 "/>

In [21]:
let mut avg_req_counts = vec![];
for lambda2 in 1..=100 {
    let rho2 = (lambda2 as f64) / &mu2;
    let dist = stationary_prob_distribution(rho1, rho2, c as u8, g as u8);
    let avg: f64 = dist.iter().enumerate().map(|(i,v)| i as f64 * v).sum();
    avg_req_counts.push((lambda2 as f32, avg as f32));
}
avg_req_counts

[(1.0, 40.20374), (2.0, 40.419373), (3.0, 40.62076), (4.0, 40.80891), (5.0, 40.984753), (6.0, 41.14919), (7.0, 41.303047), (8.0, 41.44711), (9.0, 41.582104), (10.0, 41.7087), (11.0, 41.82752), (12.0, 41.93914), (13.0, 42.044094), (14.0, 42.142864), (15.0, 42.23591), (16.0, 42.32364), (17.0, 42.406437), (18.0, 42.48465), (19.0, 42.558605), (20.0, 42.6286), (21.0, 42.694904), (22.0, 42.757774), (23.0, 42.817436), (24.0, 42.874107), (25.0, 42.927982), (26.0, 42.979248), (27.0, 43.028065), (28.0, 43.074593), (29.0, 43.118973), (30.0, 43.16134), (31.0, 43.201813), (32.0, 43.24051), (33.0, 43.27753), (34.0, 43.31298), (35.0, 43.346947), (36.0, 43.379513), (37.0, 43.41076), (38.0, 43.440758), (39.0, 43.469578), (40.0, 43.49728), (41.0, 43.52393), (42.0, 43.54958), (43.0, 43.57428), (44.0, 43.59808), (45.0, 43.621025), (46.0, 43.64316), (47.0, 43.664524), (48.0, 43.685154), (49.0, 43.705086), (50.0, 43.72435), (51.0, 43.74298), (52.0, 43.76101), (53.0, 43.778458), (54.0, 43.795357), (55.0, 43.

In [22]:
draw_chart(&avg_req_counts, "lambda2 vs Nbar")

lambda2 vs Nbar
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
40.5
 
 
 
41.0
 
 
 
41.5
 
 
 
42.0
 
 
 
42.5
 
 
 
43.0
 
 
 
43.5
 
 
 
44.0
 
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,444 41,424 47,404 53,386 59,369 65,353 71,339 77,325 83,312 89,300 95,288 101,277 107,267 113,258 119,249 125,240 131,232 137,225 143,218 149,211 156,205 162,199 168,193 174,187 180,182 186,177 192,173 198,168 204,164 210,160 216,156 222,152 228,149 234,145 240,142 246,139 252,136 258,133 264,130 270,127 277,125 283,122 289,120 295,118 301,116 307,113 313,111 319,109 325,107 331,106 337,104 343,102 349,100 355,99 361,97 367,96 373,94 379,93 385,91 391,90 398,89 404,87 410,86 416,85 422,84 428,83 434,81 440,80 446,79 452,78 458,77 464,76 470,75 476,74 482,74 488,73 494,72 500,71 506,70 512,69 519,69 525,68 531,67 537,66 543,66 549,65 555,64 561,64 567,63 573,62 579,62 585,61 591,60 597,60 603,59 609,59 615,58 621,58 627,57 634,56 "/>